# Asincronismo en Python: `async` y `await`

El asincronismo es una forma de escribir código concurrente. Nos permite ejecutar múltiples tareas que implican esperas (como llamadas a una API o consultas a una base de datos) de una manera mucho más eficiente que el código secuencial.

**La Analogía: Un Mesero Eficiente vs. un Chef Lento**
* **Código Síncrono (Secuencial):** Imagina un chef que toma tu orden, va a la cocina, prepara tu plato, te lo sirve, y **espera a que termines de comer** antes de tomarle la orden a la siguiente mesa. Es muy ineficiente.
* **Código Asíncrono:** Imagina un **mesero** (el *event loop* de `asyncio`). Toma tu orden y se la da al chef. Mientras el chef cocina (la "espera"), el mesero no se queda parado; va a otra mesa, toma otra orden, sirve bebidas, etc. Cuando el chef termina un plato, le "avisa" al mesero, y este lo recoge y lo entrega. **Es un solo mesero que gestiona múltiples tareas aprovechando los tiempos de espera.**

El asincronismo es ideal para tareas **"I/O-bound"** (limitadas por Entrada/Salida), donde el programa pasa mucho tiempo **esperando** por una respuesta.

## 1. La Sintaxis: `async`, `await` y el Event Loop

El asincronismo en Python se basa en tres componentes clave:

* **`async def` (Corrutina):** La palabra clave `async` antes de `def` convierte una función normal en una **corrutina**. Una corrutina es una función "pausable".
* **`await`**: Esta palabra clave se usa **dentro** de una corrutina para "pausar" su ejecución en un punto de espera (ej. `await asyncio.sleep(1)`) y devolver el control al *event loop* para que pueda ejecutar otras tareas.
* **`asyncio.run()`**: La función que inicia el *event loop* y ejecuta la corrutina principal.

## 2. La Magia en Acción: Ejecutando Tareas Concurrentemente

El verdadero poder del asincronismo se ve cuando ejecutamos múltiples tareas a la vez usando `asyncio.gather()`. Vamos a comparar el tiempo de ejecución.

In [ ]:
# Se debe ejecutar en un archivo .py
import asyncio
import time

# Esta es nuestra corrutina. Simula una tarea de I/O que tarda un tiempo.
async def procesar_data(id_tarea, delay):
    print(f"Iniciando tarea {id_tarea}, tardará {delay} segundos...")
    # await pausa esta función, pero permite que otras se ejecuten.
    await asyncio.sleep(delay)
    print(f"--> Tarea {id_tarea} terminada.")
    return id_tarea * 10

# --- Comparación de Tiempos ---

# 1. Forma Síncrona (Secuencial)
async def main_sincrono():
    start_time = time.time()
    # Esperamos a que cada tarea termine antes de empezar la siguiente
    await procesar_data("A", 2)
    await procesar_data("B", 3)
    print(f"Tiempo total síncrono: {time.time() - start_time:.2f} segundos.")

# 2. Forma Asíncrona (Concurrente)
async def main_asincrono():
    start_time = time.time()
    # Creamos las tareas para que se puedan ejecutar a la vez
    tareas = [
        procesar_data("A", 2),
        procesar_data("B", 3)
    ]
    # asyncio.gather() ejecuta las tareas concurrentemente
    resultados = await asyncio.gather(*tareas)
    print(f"Tiempo total asíncrono: {time.time() - start_time:.2f} segundos.")
    print(f"Resultados de las tareas: {resultados}")

# Ejecutamos ambas versiones para comparar
print("--- EJECUCIÓN SÍNCRONA ---")
asyncio.run(main_sincrono())

print("\n--- EJECUCIÓN ASÍNCRONA ---")
asyncio.run(main_asincrono())

## 3. Aplicaciones en Ciencia de Datos

Aunque el asincronismo no es para cálculos pesados (para eso usamos `multiprocessing`), es **esencial** para la recolección de datos y el despliegue de modelos.

* **Web Scraping y Recolección de Datos de APIs:**
    Este es el caso de uso #1. En lugar de hacer una petición a una URL y esperar la respuesta, puedes lanzar **cientos de peticiones a la vez**. Mientras esperas que un servidor responda, tu programa ya está enviando peticiones a otros. Esto puede reducir el tiempo de recolección de datos de horas a minutos.

* **Consultas a Bases de Datos:**
    Si necesitas ejecutar 10 consultas diferentes a una base de datos, puedes enviarlas todas de forma asíncrona. Tu programa no se quedará bloqueado esperando la primera consulta, sino que procesará los resultados a medida que la base de datos los vaya devolviendo.

* **Despliegue de Modelos de Machine Learning (APIs):**
    Cuando un modelo se despliega como una API (usando frameworks como FastAPI, que es asíncrono), necesita atender a múltiples usuarios a la vez. El asincronismo permite que tu servidor reciba una solicitud de predicción del Usuario A, la envíe al modelo, y mientras el modelo está calculando, el servidor ya está recibiendo y procesando la solicitud del Usuario B. Esto hace que la API sea increíblemente rápida y escalable.